# DSG: MADATASCIENCE 

In [3]:
import graphlab as gl
import numpy as np
import pandas as pd
import logging
import os
#logging.getLogger("requests").setLevel(logging.WARNING)
logging.disable(logging.INFO)

In [4]:
folder = os.getcwd() ; print folder 

/home/arda/Documents/DSG


In [5]:
#pd_train = pd.io.pickle.read_pickle(folder + '/data_munged/pd_train_tagged')
#pd_test = pd.io.pickle.read_pickle(folder + '/data_munged/pd_test_tagged')

pd_train = pd.io.pickle.read_pickle(folder + '/data_munged/pd_train_munged')
pd_test = pd.io.pickle.read_pickle(folder + '/data_munged/pd_test_munged')

In [6]:
pd_train['Sentence1'] = pd_train.apply(lambda x:str(x['Sentence'])+' '+str(int(x['nb_caps_word']))+' '+
                                  str(int(x['nb_!']))+' '+str(int(x['nb_,;'])),axis=1)

pd_train['Sentence2'] = pd_train.apply(lambda x:str(x['Sentence'])+' '+str(int(x['nb_caps_char']))+' '+
                                  str(int(x['nb_!']))+' '+str(int(x['nb_,;'])),axis=1)

In [7]:
sf_train = gl.SFrame(pd_train)
sf_test = gl.SFrame(pd_test)

In [11]:
def conv2ngrame(sf,nb_gram,col_name):
    sf[col_name] = gl.text_analytics.count_ngrams(sf['Sentence'], to_lower=False, n=1)
    return sf


def stopwords(sf,col_name):
    sf[col_name] = sf[col_name].dict_trim_by_keys(gl.text_analytics.stopwords(lang='en'), True)
    return sf

def tfidf(sf,col_name):
    sf[col_name] = gl.text_analytics.tf_idf(sf[col_name])['docs']
    return sf

In [12]:
#train munge
sf_train = conv2ngrame(sf_train,1,'uni')
sf_train = conv2ngrame(sf_train,2,'bi')
sf_train = conv2ngrame(sf_train,3,'tri')

#sf_train = stopwords(sf_train, 'bi')
#sf_train = stopwords(sf_train, 'uni')
#sf_train = stopwords(sf_train, 'tri')

sf_train = tfidf(sf_train,'uni')
sf_train = tfidf(sf_train,'bi')
sf_train = tfidf(sf_train,'tri')

#test munge
sf_test = conv2ngrame(sf_test,1,'uni')
sf_test = conv2ngrame(sf_test,2,'bi')
sf_test = conv2ngrame(sf_test,3,'tri')

#sf_test = stopwords(sf_test, 'bi')
#sf_test = stopwords(sf_test, 'uni')
#sf_test = stopwords(sf_test, 'tri')

sf_test = tfidf(sf_test,'uni')
sf_test = tfidf(sf_test,'bi')
sf_test = tfidf(sf_test,'tri')

In [14]:
sf_train['Author'].unique()

dtype: str
Rows: 6
['twain', 'wilde', 'austen', 'doyle', 'poe', 'shakespeare']

In [95]:
def f(r):
    out = 0
    if r=='twain':out=1;
    elif r=='doyle':out=2;
    elif r=='poe':out=3;
    elif r=='austen':out=4;
    elif r=='wilde':out=5;
    elif r=='shakespeare':out=6;
    return out

In [149]:
sf_train['label'] = sf_train['Author'].apply(lambda r: f(r))

In [16]:
print sf_train.column_names()

['Sentence', 'Author', 'lenght', 'nb_caps_char', 'nb_caps_word', 'nb_?', 'nb_,', 'nb_!', 'nb_.', 'nb_;', "nb_'", 'nb_all', 'nb_,;', 'diversity', 'Sentence1', 'Sentence2', 'uni', 'bi', 'tri']


In [18]:
F = ['Author', 'lenght', 'nb_caps_char', 'nb_caps_word', 'nb_?', 'nb_,', 'nb_!', 'nb_.', 'nb_;', "nb_'", 'nb_all', 'nb_,;', 'diversity', 'uni', 'bi', 'tri']
print F

['Author', 'lenght', 'nb_caps_char', 'nb_caps_word', 'nb_?', 'nb_,', 'nb_!', 'nb_.', 'nb_;', "nb_'", 'nb_all', 'nb_,;', 'diversity', 'uni', 'bi', 'tri']


In [180]:
for i in range(1):
    train, val = sf_train.random_split(0.8,seed=i)

    m = gl.vowpal_wabbit.create(train[F],validation_set=val target='Author', loss_function='squared', quadratic=list(),
                                l1_penalty=1e-7, l2_penalty=1e-7, bigram=True,
                                step_size=0.1, num_bits=18, verbose=False,
                                max_iterations=50, command_line_args='')

    #y_pred = m.predict(val[F]).apply(lambda r: int(np.round(r)))
    #print gl.evaluation.accuracy(y_pred, val['label'])

PROGRESS: Making predictions on training data.
0.274326939996
PROGRESS: Making predictions on training data.
0.279330572809


# ENSEMBLE BRO

In [24]:
print sf_train.column_names()

['Sentence', 'Author', 'lenght', 'nb_caps_char', 'nb_caps_word', 'nb_?', 'nb_,', 'nb_!', 'nb_.', 'nb_;', "nb_'", 'nb_all', 'nb_,;', 'diversity', 'Sentence1', 'Sentence2', 'uni', 'bi', 'tri']


In [40]:
Value	Count	Percent
austen	9,927	34.561%	
twain	6,081	21.171%	
doyle	5,534	19.267%	
wilde	4,771	16.61%	
shakespeare	2,178	7.583%	
poe	232	0.808%

Canvas is accessible via web browser at the URL: http://localhost:47779/index.html


In [59]:
F = ['lenght', 'nb_caps_char', 'nb_caps_word', 'nb_?', 'nb_,', 'nb_!', 'nb_.', 'nb_;', "nb_'", 'nb_all', 'nb_,;', 'diversity']
print F

['lenght', 'nb_caps_char', 'nb_caps_word', 'nb_?', 'nb_,', 'nb_!', 'nb_.', 'nb_;', "nb_'", 'nb_all', 'nb_,;', 'diversity']


In [62]:
weights = {'austen': 35, 'twain': 21, 'doyle': 20, 'wilde': 17, 'shakespeare': 8, 'poe': 2}

for i in range(1):
    train, val = sf_train.random_split(0.8, seed=i)
    m = gl.classifier.boosted_trees_classifier.create(train, validation_set=val,
                                                       features=F, target='Author',
                                                       max_iterations=200,
                                                      max_depth=10,min_child_weight=0.1,step_size=0.3,
                                                       class_weights=weights,
                                                       verbose=True)

PROGRESS: Boosted trees classifier:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 23040
PROGRESS: Number of classes           : 6
PROGRESS: Number of feature columns   : 12
PROGRESS: Number of unpacked features : 12
PROGRESS: Starting Boosted Trees
PROGRESS: --------------------------------------------------------
PROGRESS:   Iter      Accuracy          Elapsed time
PROGRESS:         (training) (validation)
PROGRESS:      0   5.987e-01   5.461e-01        0.14s
PROGRESS:      1   6.056e-01   5.487e-01        0.24s
PROGRESS:      2   6.112e-01   5.506e-01        0.35s
PROGRESS:      3   6.152e-01   5.487e-01        0.46s
PROGRESS:      4   6.198e-01   5.482e-01        0.57s
PROGRESS:      5   6.232e-01   5.494e-01        0.68s
PROGRESS:      6   6.257e-01   5.492e-01        0.79s
PROGRESS:      7   6.289e-01   5.506e-01        0.90s
PROGRESS:      8   6.334e-01   5.476e-01        1.01s
PROGRESS:      9   6.365e-01   5.452e-01  

In [52]:
from sklearn.metrics import accuracy_score

y_pred = list(m.predict(val))
gt = list(val['Author'])

print accuracy_score(y_pred,gt)

0.753827203942


# KNN

In [82]:
distances = ['squared_euclidean', 'manhattan', 'jaccard', 'weighted_jaccard', 'cosine','dot_product']
for i in distances:

    train, val = sf_train.random_split(0.8)
    F = ['uni','bi','tri']
    m = gl.nearest_neighbor_classifier.create(train, target='Author',features=F, distance=i, verbose=True)

    print m.evaluate(val)



Starting nearest neighbors model construction...
PROGRESS: Starting to query
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | Query points | # Pairs | % Complete. | Elapsed Time |
PROGRESS: +--------------+---------+-------------+--------------+
PROGRESS: | 0            | 5769    | 0           | 25.835ms     |
PROGRESS: | 999          | 2.3e+07 | 17.25       | 1.02s        |
PROGRESS: | 1981         | 4.5e+07 | 34.25       | 2.02s        |
PROGRESS: | 2958         | 6.8e+07 | 51.25       | 3.02s        |
PROGRESS: | 3811         | 8.7e+07 | 66          | 4.02s        |
PROGRESS: | 4785         | 1.1e+08 | 82.75       | 5.02s        |
PROGRESS: | 5570         | 1.3e+08 | 96.5        | 6.02s        |
PROGRESS: | Done         |         | 100         | 6.64s        |
PROGRESS: +--------------+---------+-------------+--------------+
{'confusion_matrix': Columns:
	target_label	str
	predicted_label	str
	count	int

Rows: 25

Data:
+--------------+-----------------+

# Submit Kaggle

In [214]:
m.save(folder +'/gl_GBT')

In [53]:
y_pred = m.predict(sf_test)

In [55]:
submit = gl.SFrame()
submit['id']=sf_test['Id']
submit['label']=y_pred

In [56]:
submit.save(folder + '/results/out1.csv', format='csv')